In [54]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt

In [8]:
path = '/mnt/c/Users/User/Desktop/savgol_filter_lc_data/'
files=[]

for filepath in glob.glob(path + '*.csv'):
    #print(filepath)
    df = pd.read_csv(filepath)
    print(len(df['filter_mag']))
    files.append(str(filepath))

986
1282
1282
1282
1282
1282
1266
1184
16781
1282
1266
1282
1282
1282
1282
1245
1282
1282
1282
1282
1282
1266
1245
1282
1266
1266
1282
1184
1266
1282
1282
1266
1266
1245
1245
1282
1282
1282
1228
1282
1245
1282
1282
1282
1282
1282
1282
1282
1184
1174
1046
1025
1282
1184
1282
1282
1282
1046
986
1196
1228
980
980
986
1282
1266
1184
1188
1188
1188
986
986
1266
1184
1282
1266
1282
1184
1266
1266
1282
1184
1282
1266
1205
1086
1086
1086
1266
1266
1266
1184
1282
1245
1184
842
1046
1228
1282
1196
1228
1282
1282
1282
1282
1282
1282
1282
1245
1245
1282
1282
1282
1282
1228
1266
1245
1282
1282
1241
880
1184
1266
1282
1282
1282
1245
1282
1282
986
986
1282
1228
1266
1282
986
1245
1228
1228
1205
1086
1245
986
986
1282
1282
1282
1282
1266
1282
1282
1282
1282
1245
1282
1282
1282
1282
1282
1245
1245
1282
1282
1282
1266
1282
1282
1282
1245
1282
1266
1266
1282
1266
1282
1282
1282
1245
1245
1148
1148
1282
1148
1148
1282
1282
1245
1148
1282
1148
1205
1266
1282
1282
1282
1245
1282
1245
1282
1282
1282
1266
124

In [3]:
files[0]


'/mnt/c/Users/User/Desktop/mulens_no_noise_data/t_0=3500.u_0=-0.055.t_E=1.1011.s=1.0.q=0.1.alpha=162.rho=0.1.csv'

In [12]:
for filepath in glob.glob(path + '*.csv'):
    df = pd.read_csv(filepath)
    if len(df['filter_mag'])>10000:
        files.remove(filepath)

In [13]:
for filepath in files:
    #print(filepath)
    df = pd.read_csv(filepath)
    print(len(df['filter_mag']))
    

986
1282
1282
1282
1282
1282
1266
1184
1282
1266
1282
1282
1282
1282
1245
1282
1282
1282
1282
1282
1266
1245
1282
1266
1266
1282
1184
1266
1282
1282
1266
1266
1245
1245
1282
1282
1282
1228
1282
1245
1282
1282
1282
1282
1282
1282
1282
1184
1174
1046
1025
1282
1184
1282
1282
1282
1046
986
1196
1228
980
980
986
1282
1266
1184
1188
1188
1188
986
986
1266
1184
1282
1266
1282
1184
1266
1266
1282
1184
1282
1266
1205
1086
1086
1086
1266
1266
1266
1184
1282
1245
1184
842
1046
1228
1282
1196
1228
1282
1282
1282
1282
1282
1282
1282
1245
1245
1282
1282
1282
1282
1228
1266
1245
1282
1282
1241
880
1184
1266
1282
1282
1282
1245
1282
1282
986
986
1282
1228
1266
1282
986
1245
1228
1228
1205
1086
1245
986
986
1282
1282
1282
1282
1266
1282
1282
1282
1282
1245
1282
1282
1282
1282
1282
1245
1245
1282
1282
1282
1266
1282
1282
1282
1245
1282
1266
1266
1282
1266
1282
1282
1282
1245
1245
1148
1148
1282
1148
1148
1282
1282
1245
1148
1282
1148
1205
1266
1282
1282
1282
1245
1282
1245
1282
1282
1282
1266
1245
1282

In [41]:
def div_by_2(integer):
    'returns divisor=[floor, ceiling] '
    from math import floor, ceil
    divisors = np.zeros(2)
    if (integer%2)==0:
        divisors[0] = int(integer//2)
        divisors[1] = int(integer//2)
    else:
        divisors[0] = int(floor(integer/2))
        divisors[1] = int(ceil(integer/2))
    divisors = [int(item) for item in divisors]
    return (divisors)

In [77]:
def trimmed_data(dataframe, length):
    n_left = div_by_2(abs(len(dataframe)-length))[0]
    n_right = div_by_2(abs(len(dataframe)-length))[1]
    if len(dataframe)>length:
        df = dataframe.loc[n_left:len(dataframe)-n_right]
    else:
        elem_left = pd.DataFrame(np.ones(div_by_2(abs(len(dataframe)-length))[0])*dataframe.iloc[0])
        elem_right = pd.DataFrame(np.ones(div_by_2(abs(len(dataframe)-length))[1])*dataframe.iloc[-1])
        df = pd.concat([dataframe, elem_right])
        df = pd.concat([elem_left, df])
    return(df)

In [4]:
def avg_clean(arr, n):
    # Average of every nth element in array
    remainder = len(arr) % n
    if remainder == 0:
        avg = np.mean(arr.reshape(-1, n), axis=1)
        avg = np.repeat(avg, n)
        return avg
    else:
        avg_head = np.mean(arr[:-remainder].reshape(-1, n), axis=1)
        avg_tail = np.mean(arr[-remainder:])
        avg_head = np.repeat(avg_head, n)
        avg_tail = np.repeat(avg_tail, remainder)
        return np.append(avg_head, avg_tail)

def normalization(arr):
    # Normalizes data via x_norm = (x-min)/(max-min)
    normed_arr = np.zeros(len(arr))
    for i in range(len(arr)):
        normed_arr[i] = (arr[i]-min(arr))/(max(arr)-min(arr))
    return(normed_arr)

In [5]:
path = '/mnt/c/Users/User/Desktop/norm_savgol_filter_lc/'
for i in range(len(files)):
    df = pd.read_csv(files[i])
    if len(df['mag'])>1500:
        clean = avg_clean(np.asarray(df['mag']), n=7)
        clean_normed = normalization(clean)
        clean_normed_csv = pd.DataFrame(data=clean_normed)
        np.savetxt(path + 'cleaned_normed_' + os.path.basename(files[i]), X=clean_normed_csv)
    else:
        clean_normed = normalization(np.asarray(df['mag']))
        clean_normed_csv = pd.DataFrame(data=clean_normed)
        np.savetxt(path + 'cleaned_normed_' + os.path.basename(files[i]), X=clean_normed_csv)

In [84]:
df = pd.read_csv(files[1])
df2 = trimmed_data(df['filter_mag'], 1000)
#df = normalization(df2)
df2

141     8.831638
142     8.833542
143     8.834849
144     8.836667
145     8.838647
          ...   
1137    9.025864
1138    9.027099
1139    9.028142
1140    9.029069
1141    9.030382
Name: filter_mag, Length: 1001, dtype: float64

In [88]:
path = '/mnt/c/Users/User/Desktop/norm_savgol_filter_lc/'
for i in range(len(files)):
    df = pd.read_csv(files[i])
    df2 = trimmed_data(df['filter_mag'], 1000)
    df = normalization(df2.values)
    
    np.savetxt(path + 'trimmed_normed_' + os.path.basename(files[i]), X=df)